In [ ]:
import imagepers
import numpy as np
from read_chroma import read_chromato_and_chromato_cube
import matplotlib.pyplot as plt
import pandas as pd
import read_chroma
import peak_detection
import baseline_correction
import skimage
import ipywidgets as widgets

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
mod_time = 1.7

In [ ]:
# chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)


In [ ]:
chromato_obj = read_chroma.read_chroma(file_path, mod_time)
tic_chromato, time_rn, spectra_obj = chromato_obj

chromato_tic_preprocessed = baseline_correction.chromato_reduced_noise(tic_chromato)
dynamic_threshold_fact = 0.1

In [ ]:
def analyze_persistence_distribution(chromato):
    """
    Analyze the distribution of persistence values in the chromatogram.
    
    Parameters
    ----------
    chromato : ndarray
        2D chromatogram
        
    Returns
    -------
    tuple
        (persistent_groups, df_stats)
    """
    # Calculate persistence for all peaks
    persistent_groups = imagepers.persistence(chromato)
    
    # Extract persistence values
    persistence_values = [pers_val for _, _, pers_val, _ in persistent_groups]
    
    # Calculate statistics
    max_intensity = np.max(chromato)
    stats = {
        'Total peaks': len(persistent_groups),
        'Min persistence': min(persistence_values) if persistence_values else 0,
        'Max persistence': max(persistence_values) if persistence_values else 0,
        'Mean persistence': np.mean(persistence_values) if persistence_values else 0,
        'Median persistence': np.median(persistence_values) if persistence_values else 0,
        'Min persistence (relative)': min(persistence_values)/max_intensity if persistence_values else 0,
        'Max persistence (relative)': max(persistence_values)/max_intensity if persistence_values else 0,
        'Mean persistence (relative)': np.mean(persistence_values)/max_intensity if persistence_values else 0,
        'Median persistence (relative)': np.median(persistence_values)/max_intensity if persistence_values else 0
    }
    
    # Create DataFrame for easier analysis
    df_stats = pd.DataFrame([stats])
    
    # Create data for percentile analysis
    percentiles = list(range(0, 101, 5))
    percentile_values = np.percentile(persistence_values, percentiles) if persistence_values else np.zeros_like(percentiles)
    relative_percentiles = percentile_values / max_intensity
    
    # Plot the distribution
    plt.figure(figsize=(12, 10))
    
    # Persistence histogram
    plt.subplot(2, 1, 1)
    plt.hist(persistence_values, bins=50, alpha=0.7)
    plt.axvline(np.median(persistence_values), color='r', linestyle='--', 
               label=f'Median: {np.median(persistence_values):.2f}')
    plt.xlabel('Persistence Value')
    plt.ylabel('Count')
    plt.title('Distribution of Persistence Values')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Relative persistence percentiles
    plt.subplot(2, 1, 2)
    plt.plot(percentiles, relative_percentiles, 'o-')
    plt.xlabel('Percentile')
    plt.ylabel('Relative Persistence (fraction of max intensity)')
    plt.title('Percentiles of Relative Persistence Values')
    plt.grid(True, alpha=0.3)
    
    # Highlight some key percentiles
    key_percentiles = [50, 75, 90, 95, 100]
    key_indices = [percentiles.index(p) for p in key_percentiles]
    key_values = [relative_percentiles[i] for i in key_indices]
    
    plt.scatter([percentiles[i] for i in key_indices], 
               [relative_percentiles[i] for i in key_indices], 
               color='red', s=100, zorder=5)
    
    for p, v in zip(key_percentiles, key_values):
        plt.annotate(f'{p}%: {v:.5f}', 
                    (p, v), 
                    textcoords="offset points", 
                    xytext=(0, 10), 
                    ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Print key percentiles
    print("Key percentiles for min_persistence selection:")
    for p, v in zip(key_percentiles, key_values):
        print(f"{p}th percentile: {v:.5f}")
    
    print("\nSuggested min_persistence values:")
    print(f"Conservative (keeps ~50% of peaks): {key_values[0]* 100:.3f}")
    print(f"Balanced (keeps ~25% of peaks): {key_values[1] * 100:.3f}")
    print(f"Stringent (keeps ~10% of peaks): {key_values[2] * 100:.3f}")
    print(f"Very stringent (keeps ~5% of peaks): {key_values[3] * 100:.3f}")
    # print(f"Extremely stringent (keeps ~1% of peaks): {key_values[4]:.5f}")
    
    return persistent_groups, df_stats

persistent_groups, stats = analyze_persistence_distribution(chromato_tic_preprocessed)


In [ ]:
def evaluate_min_persistence(chromato_obj, chromato_cube=None, mode="tic", 
                            persistence_values=None, threshold_abs=0.01):
    """
    Evaluate different min_persistence values and visualize the results.
    
    Parameters
    ----------
    chromato_obj : tuple
        Tuple of (chromato, time_rn)
    chromato_cube : ndarray, optional
        3D chromatogram cube, required for mass_per_mass mode
    mode : str
        'tic' or 'mass_per_mass'
    persistence_values : list
        List of persistence values to evaluate
    threshold_abs : float
        Absolute intensity threshold
        
    Returns
    -------
    dict
        Dictionary mapping persistence values to peak counts
    """
    if persistence_values is None:
        # Default range of persistence values to evaluate
        persistence_values = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3]
    
    # chromato, time_rn = chromato_obj
    results = {}
    
    # Process each persistence value
    for persistence in persistence_values:
        peaks = peak_detection.pers_hom(chromato_obj, persistence, threshold_abs=threshold_abs, 
                        mode=mode, chromato_cube=chromato_cube)
        results[persistence] = peaks
        print(f"Min persistence: {persistence:.4f} -> {len(peaks)} peaks detected")
    
    # Plot number of peaks vs persistence
    plt.figure(figsize=(10, 6))
    x_values = list(results.keys())
    y_values = [len(peaks) for peaks in results.values()]
    
    plt.plot(x_values, y_values, 'o-', linewidth=2)
    plt.xlabel('Min Persistence Value')
    plt.ylabel('Number of Peaks Detected')
    plt.title('Effect of Min Persistence on Peak Detection')
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    
    # Calculate rate of change between consecutive points
    if len(x_values) > 1:
        x_midpoints = [(x_values[i] + x_values[i+1])/2 for i in range(len(x_values)-1)]
        rates = [abs((y_values[i+1] - y_values[i])/(x_values[i+1] - x_values[i])) 
                for i in range(len(x_values)-1)]
        
        # Plot rate of change on secondary axis
        ax2 = plt.gca().twinx()
        ax2.plot(x_midpoints, rates, 'r--', linewidth=1, alpha=0.7)
        ax2.set_ylabel('Rate of Change (peaks/persistence)', color='r')
        ax2.tick_params(axis='y', labelcolor='r')
    
    plt.tight_layout()
    plt.show()
    
    return results

results = evaluate_min_persistence((chromato_tic_preprocessed, time_rn), mode="tic", 
                                  persistence_values=[0.001, 0.005, 0.01, 0.02, 0.05, 0.1])

In [ ]:
from matplotlib.widgets import Slider, Button

In [ ]:
def visualize_peaks_with_slider(chromato_obj, chromato_cube=None, mode="tic",
                              threshold_abs=0.01, init_persistence=0.02):
    """
    Interactive visualization with a slider to adjust persistence value using Jupyter widgets.
    Includes loading indicator and ensures only one graph is displayed at a time.
    
    Parameters
    ----------
    chromato_obj : tuple
        Tuple of (chromato, time_rn)
    chromato_cube : ndarray, optional
        3D chromatogram cube, required for mass_per_mass mode
    mode : str
        'tic' or 'mass_per_mass'
    threshold_abs : float
        Absolute intensity threshold
    init_persistence : float
        Initial persistence value
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    import peak_detection  # Assurez-vous que ce module est importé correctement
    import time
    
    # Vérification des importations pour débugger
    print("Modules importés avec succès")
    
    chromato, time_rn = chromato_obj
    
    # Clear any existing plots to avoid accumulation
    plt.close('all')
    
    # Create widgets for the entire app
    loading_status = widgets.HTML(
        value='<div style="display: flex; align-items: center;"><div style="color: green; font-weight: bold;">Prêt</div></div>'
    )
    timing_info = widgets.HTML(value="Temps de calcul: -- secondes")
    peak_info = widgets.HTML(value="<div style='font-weight: bold;'>Nombre de pics: --</div>")
    
    # Output widget pour contenir UNIQUEMENT la visualisation
    graph_output = widgets.Output()
    
    # Create slider widget
    slider = widgets.FloatSlider(
        value=init_persistence,
        min=0.001,
        max=0.3,
        step=0.001,
        description='Min Persistence:',
        continuous_update=False,  # Update only when released
        layout=widgets.Layout(width='70%')
    )
    
    # Create button for manual update
    update_button = widgets.Button(
        description='Mettre à jour',
        button_style='primary',
        tooltip='Cliquez pour mettre à jour le graphique'
    )
    
    # Create layout before defining callback functions
    status_row = widgets.HBox([loading_status, timing_info])
    controls = widgets.VBox([slider, update_button, status_row, peak_info])
    app = widgets.VBox([controls, graph_output])
    
    # Display the app once at the beginning - this is important!
    display(app)
    
    # Fonction pour mettre à jour l'indicateur de chargement
    def update_loading_status(is_loading=True):
        if is_loading:
            loading_status.value = '''
            <div style="display: flex; align-items: center;">
                <div style="border: 4px solid rgba(0, 0, 0, 0.1); border-left-color: blue; 
                     border-radius: 50%; width: 20px; height: 20px; margin-right: 10px;
                     animation: spin 1s linear infinite;"></div>
                <div style="color: blue; font-weight: bold;">Calcul en cours...</div>
            </div>
            <style>
                @keyframes spin {
                    0% { transform: rotate(0deg); }
                    100% { transform: rotate(360deg); }
                }
            </style>
            '''
        else:
            loading_status.value = '<div style="display: flex; align-items: center;"><div style="color: green; font-weight: bold;">Calcul terminé</div></div>'
            # Après une courte pause, remettre à "Prêt"
            time.sleep(0.5)  # Pause courte pour s'assurer que le message est visible
            loading_status.value = '<div style="display: flex; align-items: center;"><div style="color: green; font-weight: bold;">Prêt</div></div>'
    
    # Fonction de mise à jour qui sera appelée quand le slider change
    def update_plot(persistence):
        # Mettre à jour le statut: chargement
        update_loading_status(True)
        
        # Mesurer le temps d'exécution
        start_time = time.time()
        
        # Clear l'output AVANT d'afficher une nouvelle figure
        with graph_output:
            clear_output(wait=True)
            plt.close('all')  # Fermer toutes les figures existantes
            
            # Create a new figure
            fig, ax = plt.subplots(figsize=(12, 6))
            
            # Display the chromatogram
            im = ax.imshow(chromato, cmap='viridis', aspect='auto', interpolation='nearest')
            plt.colorbar(im, ax=ax, label='Intensité')
            
            # Calculate peaks with current persistence
            try:
                peaks = peak_detection.pers_hom(chromato_obj, persistence, threshold_abs=threshold_abs,
                                              mode=mode, chromato_cube=chromato_cube)
                
                # Update peak count
                peak_info.value = f'<div style="font-weight: bold;">Nombre de pics détectés: {len(peaks)}</div>'
                
                # Plot peaks
                if len(peaks) > 0:
                    ax.scatter(peaks[:, 1], peaks[:, 0], color='red', marker='x', s=100, linewidths=2)
                
                # Set title
                ax.set_title(f'Detected Peaks (min_persistence={persistence:.4f}, {len(peaks)} peaks)')
                
            except Exception as e:
                # Handle errors
                print(f"Erreur lors de la détection des pics: {str(e)}")
                peak_info.value = f'<div style="color: red; font-weight: bold;">Erreur: {str(e)}</div>'
                ax.set_title(f'Erreur lors de la détection des pics')
            
            # Show plot with tight layout
            plt.tight_layout()
            plt.show()
        
        # Calculer et afficher le temps écoulé
        elapsed_time = time.time() - start_time
        timing_info.value = f"Temps de calcul: {elapsed_time:.3f} secondes"
        
        # Mettre à jour le statut: terminé
        update_loading_status(False)
    
    # Connect callbacks
    def on_value_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            update_plot(change['new'])
    
    slider.observe(on_value_change, names='value')
    
    def on_button_click(b):
        update_plot(slider.value)
    
    update_button.on_click(on_button_click)
    
    # Initial update - après avoir affiché l'app
    update_plot(init_persistence)
    
    return slider, graph_output

# Utilisation:
slider, output = visualize_peaks_with_slider((chromato_tic_preprocessed, time_rn), mode="tic", threshold_abs=0.01)